# Build generative AI-powered customer support agent using Amazon Strands Agent and Amazon Bedrock


## Overview

This notebook demonstrates the construction of an advanced customer support agent using Amazon Strands Agent and Amazon Bedrock. We focus on developing tools to automate various customer support tasks, enhancing efficiency and response times.

### Key Features

- 🔍 Information Lookup (retrieve)
- 📈 Service Limit Increase (increase_limit)
- 👀 View Current Limits (view_customer_limits)
- 💰 Process Billing Adjustments (adjust_billing)
- 📊 Check Billing Status (view_customer_billing)
- 🛠️ Payment Issue Resolution (resolve_payment_issue)
- 💸 Refund Processing (process_refund)
- 📝 Customer Feedback Management (manage_customer_feedback)

## Solution Components

- Amazon Strands Agent for tool automation
- SQLite integration for operational data management
- Amazon Bedrock Knowledge Base for comprehensive troubleshooting information

## Prerequisites

1. **Infrastructure Setup**
   - Deployment of CloudFormation template creating:
     - S3 bucket
     - VPC with public and private subnets
     - IAM roles for SageMaker and Bedrock Knowledge Base
     - OpenSearch Serverless Collection
     - SageMaker domain and user profiles

2. **Knowledge Base Setup**
   - Execute `KBTroublehsooting.ipynb` to:
     - Create vector index in the collection
     - Establish Bedrock Knowledge Base
     - Set up data source and ingestion job

3. **Access Requirements**
   - Bedrock Models: Anthropic Claude Sonnet 3
   - Python packages: boto3, requests_aws4auth, strands-agents, strands-agents-tools, streamlit
   - SQLite database

## Purpose

This solution provides a practical, versatile framework for support teams to create customized agents tailored to specific applications and products. It offers an intuitive approach to support agent development, focusing on automating manual tasks and improving customer service efficiency.

---

**Note:** Ensure all prerequisites are met before proceeding with this notebook.


In [2]:
pip install strands-agents strands-agents-tools

Note: you may need to restart the kernel to use updated packages.


In [3]:
"""
Import customer data from customer_supportdata.json

Purpose:
- Load initial customer-specific data into database tables
- File contains structured data for:
    * Customer profiles
    * Service limits
    * Billing information
    * Customer feedback
    * Support history

Note: This is a one-time import to populate the database with baseline data
"""
from limit_db import CustomerSupportDB
db = CustomerSupportDB()
db.reset_database()
db.import_json_data('customer_supportdata.json')

Existing database removed: customer_support.db
Tables created successfully!
New database created with fresh tables
Data imported successfully!


In [4]:
"""
Database Verification Function

Purpose:
- Validates table contents after agent performs operations
- Ensures data integrity and correct execution of changes
- Retrieves current state of specified table

Use cases:
- Post-operation validation
- Debugging agent actions
- Monitoring data changes
"""



from limit_db import CustomerSupportDB
def verify_database_update():
    db = CustomerSupportDB()
    
    print("=== Database Verification ===")
    
    # Get summary of all tables
    summary = db.verify_database()
    
    if summary:
        print("\nDatabase Summary:")
        for table_name, data in summary.items():
            print(f"\n{table_name.upper()}:")
            print(f"Total records: {data['record_count']}")
            if data['sample_record']:
                print("Sample record:")
                for key, value in data['sample_record'].items():
                    print(f"  {key}: {value}")
    
    # Print all data in tables
    print("\n=== Complete Database Contents ===")
    db.print_all_data()
# Run verification
verify_database_update()

=== Database Verification ===

Database Summary:

LIMITS:
Total records: 3
Sample record:
  request_id: 1001
  customer_id: CUST_001
  feature_id: API_CALLS
  feature_name: API Rate Limit
  default_limit: 1000
  current_limit: 1000
  requested_limit: 2500
  status: PENDING
  request_date: 2024-06-12T09:30:00
  last_updated: 2024-06-12T09:30:00

BILLING:
Total records: 4
Sample record:
  billing_id: 1001
  customer_id: CUST_001
  invoice_date: 2024-06-01
  amount: 500.0
  status: PAID
  payment_method: Credit Card ending in 1234
  last_four: 1234

REFUNDS:
Total records: 3
Sample record:
  refund_id: 2001
  customer_id: CUST_003
  billing_id: 1003
  request_date: 2024-06-05
  amount: 1000.0
  status: PROCESSING

FEEDBACK:
Total records: 4
Sample record:
  feedback_id: 3001
  customer_id: CUST_001
  service: Compute
  rating: 4
  comment: Great performance, but setup was a bit complex.
  date: 2024-06-10

=== Complete Database Contents ===

=== Limit Management Records ===
   request_id 

In [5]:
"""
Support Agent Initialization and Configuration

This cell sets up the core components of the customer support agent:

1. Import Dependencies:
   - Strands Agent and BedrockModel for agent functionality
   - Custom tools for various support operations

2. Configuration:
   - Sets up region and Knowledge Base ID
   - Initializes Bedrock model (Claude 3 Sonnet)

3. System Prompt:
   - Defines agent's role and responsibilities
   - Outlines handling procedures for different request types
   - Specifies usage of knowledge base for technical support

4. Tool Integration:
   - Combines all support tools into unified agent interface:
     * Information retrieval (knowledge base)
     * Limit management
     * Billing operations
     * Payment processing
     * Customer feedback handling

5. Testing Function:
   - Validates agent's response capabilities
   - Tests knowledge base integration

Note: Ensure Knowledge Base ID is properly configured in kb_id.txt
"""


import os
from strands import Agent
from strands.models import BedrockModel
from strands_tools import retrieve
from limit_increase_tool import increase_limit
from view_limits_tool import view_customer_limits
from billing_adjust_tool import adjust_billing
from view_billing_tool import view_customer_billing
from payment_issue_tool import resolve_payment_issue
from refund_processing_tool import process_refund
from customer_feedback_tool import manage_customer_feedback

# Get the KB ID
def get_kb_id():
    with open('kb_id.txt', 'r') as f:
        return f.read().strip()

# Configuration
REGION = 'us-east-1'
KB_ID = get_kb_id()
os.environ['KNOWLEDGE_BASE_ID'] = KB_ID
# Initialize the Bedrock model
model = BedrockModel(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    region_name=REGION
)



system_prompt = """You are a customer support agent for a cloud services company.
Your tasks include:
1. Assisting customers with limit increase requests
2. Providing information about current limits
3. Adjusting billing amounts when necessary
4. Viewing customer billing records
5. Resolving payment issues
6. Processing refund requests and providing refund status updates
7. Collecting and managing customer feedback
8. Answering technical and troubleshooting questions using the knowledge base

For technical support and troubleshooting:
- First check the knowledge base using the retrieve tool
- Provide clear, step-by-step solutions from the documentation
- If the knowledge base doesn't have relevant information, clearly state that

For all other requests:
- Use the appropriate tools to handle customer requests
- Always verify customer information before making changes
- Provide clear confirmation of actions taken

Use the provided tools to help customers with their requests."""

# Initialize the agent with all tools including retrieve
agent = Agent(
    model=model,
    system_prompt=system_prompt,
    tools=[
        retrieve,
        increase_limit,
        view_customer_limits,
        adjust_billing,
        view_customer_billing,
        resolve_payment_issue,
        process_refund,
        manage_customer_feedback
    ],
)

def test_tools():
    """Test various tools and capabilities"""
    
    

    # Test knowledge base query
    print("\nTesting knowledge base query:")
    response = agent("I'm experiencing slow upload speeds to storage. What could be the issue?")
    print(f"Agent: {response}")

    

if __name__ == "__main__":
    test_tools()



Testing knowledge base query:

Tool #1: retrieve


The knowledge base query did not return any relevant results for troubleshooting slow upload speeds to storage. However, here are some common potential causes and steps to investigate:

1. Network Bandwidth: Check if you are getting the expected network bandwidth from your internet service provider. Slow uploads could be caused by network congestion or bandwidth throttling.

2. File Size: Large file sizes can take longer to upload, especially over slower network connections. Try uploading a smaller file to test.

3. Concurrent Transfers: If you are uploading multiple files concurrently, it can impact upload speeds for each transfer. Try uploading files one at a time.

4. Server Load: If many users are uploading files at the same time, it can overload the storage service and slow down uploads. This would be a service provider issue.

5. Geographic Distance: Greater geographic distance between you and the storage service can increase la

In [6]:
response = agent("There seems to be an error on my last bill. I'm customer CUST_001, and the billing ID is 1001. Can you adjust the amount to $450 instead of $500?")


Certainly, let me adjust that billing amount for you.
Tool #2: adjust_billing


I have updated the billing amount for ID 1001 from $500 to $450 for customer CUST_001. Please let me know if you need any further assistance with your billing.

In [7]:
response = agent("I need to increase my API call limit. My customer ID is CUST_001 and I want to increase the API_CALLS limit to 3000.")


Got it, let me increase the API call limit for your account.
Tool #3: increase_limit


Your request to increase the API_CALLS limit to 3000 for customer CUST_001 has been submitted successfully. The previous limit was 1000. This request is currently pending review and approval from our operations team.

Please allow up to 24-48 hours for the new 3000 API call limit to be provisioned on your account. I'll notify you once the limit increase is complete. Let me know if you need anything else!

In [8]:
# Run verification
verify_database_update()

=== Database Verification ===

Database Summary:

LIMITS:
Total records: 4
Sample record:
  request_id: 1001
  customer_id: CUST_001
  feature_id: API_CALLS
  feature_name: API Rate Limit
  default_limit: 1000
  current_limit: 1000
  requested_limit: 2500
  status: PENDING
  request_date: 2024-06-12T09:30:00
  last_updated: 2024-06-12T09:30:00

BILLING:
Total records: 4
Sample record:
  billing_id: 1001
  customer_id: CUST_001
  invoice_date: 2024-06-01
  amount: 450.0
  status: PAID
  payment_method: Credit Card ending in 1234
  last_four: 1234

REFUNDS:
Total records: 3
Sample record:
  refund_id: 2001
  customer_id: CUST_003
  billing_id: 1003
  request_date: 2024-06-05
  amount: 1000.0
  status: PROCESSING

FEEDBACK:
Total records: 4
Sample record:
  feedback_id: 3001
  customer_id: CUST_001
  service: Compute
  rating: 4
  comment: Great performance, but setup was a bit complex.
  date: 2024-06-10

=== Complete Database Contents ===

=== Limit Management Records ===
   request_id 

In [10]:
response = agent("Can you tell me the status of my refund request? I'm customer CUST_001, and it's for billing ID 1001.")

Sure, let me check on the status of your refund request.
Tool #5: process_refund


Based on the information retrieved, here is the status of your refund request:

Refund ID: 2002
Customer ID: CUST_001  
Billing ID: 1001
Refund Amount: $100.00
Request Date: 2024-06-10
Status: APPROVED

Your refund request for $100 against billing ID 1001 was approved. The refund amount of $100 should be credited back to your account or original payment method shortly.

Please let me know if you need any other details regarding this refund!

In [11]:
response = agent("I want to submit a review for your Storage service. I'm customer CUST_002, and I give it 5 stars. The reliability and ease of use are excellent.")

Thank you for taking the time to provide feedback on our Storage service, I really appreciate it.
Tool #6: manage_customer_feedback


Thank you for the 5-star review! I'm delighted to hear that you find our Storage service reliable and easy to use. Customer satisfaction is our top priority, so feedback like yours lets us know we're on the right track.

Your comments have been recorded with Feedback ID 3005. We really value input from customers like you to help us continue enhancing our products and services.

Please let me know if there's anything else I can assist with regarding our Storage service or other offerings!

In [12]:
# Run verification
verify_database_update()

=== Database Verification ===

Database Summary:

LIMITS:
Total records: 4
Sample record:
  request_id: 1001
  customer_id: CUST_001
  feature_id: API_CALLS
  feature_name: API Rate Limit
  default_limit: 1000
  current_limit: 1000
  requested_limit: 2500
  status: PENDING
  request_date: 2024-06-12T09:30:00
  last_updated: 2024-06-12T09:30:00

BILLING:
Total records: 4
Sample record:
  billing_id: 1001
  customer_id: CUST_001
  invoice_date: 2024-06-01
  amount: 450.0
  status: PAID
  payment_method: Credit Card ending in 1234
  last_four: 1234

REFUNDS:
Total records: 3
Sample record:
  refund_id: 2001
  customer_id: CUST_003
  billing_id: 1003
  request_date: 2024-06-05
  amount: 1000.0
  status: PROCESSING

FEEDBACK:
Total records: 5
Sample record:
  feedback_id: 3001
  customer_id: CUST_001
  service: Compute
  rating: 4
  comment: Great performance, but setup was a bit complex.
  date: 2024-06-10

=== Complete Database Contents ===

=== Limit Management Records ===
   request_id 

In [13]:
response = agent("Our database connection keeps failing. What should I check?")

Let me check the knowledge base for any relevant information on troubleshooting database connection failures.
Tool #7: retrieve


Unfortunately, the knowledge base did not return any specific guidance for troubleshooting database connection issues. However, here are some common steps you can take:

1. **Check Network Connectivity**
Ensure you have a stable network connection between your application and the database server. Try pinging the database server IP address to verify network reachability.

2. **Verify Database Server Status** 
Confirm the database server is up and running. Check server logs for any errors or restart the database service if needed.

3. **Check Firewall Rules**
Make sure firewall rules allow incoming connections from your application to the database server on the proper port (e.g. 3306 for MySQL).

4. **Review Connection String**
Verify the connection string parameters are correct (server address, port, database name, username, password).

5. **Check Connection 

# Streamlit Installation and Front-End Development

## Overview
This section covers the setup and implementation of our customer support interface using Streamlit.

## Installation
```bash
pip install streamlit
```

## Components
- Interactive chat interface
- Customer request forms
- Response displays

---


In [12]:
pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 89.2 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 123.1 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [13]:
"""
# Streamlit Setup and Interface Development
-----------------------------------------

Purpose:
Build an interactive web interface for the customer support agent

Components:
1. Chat Interface
   - Message history display
   - User input field
   - Response formatting

2. Support Tools
   - Limit increase request form
   - Billing inquiry interface
   - Technical support section
   - Feedback collection form

Dependencies:
- streamlit
- st_chat_message
- session_state handlers

Usage:
Run with: streamlit run app.py
"""

!streamlit run app.py --server.runOnSave true




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8502
  Network URL: http://169.255.254.1:8502
  External URL: http://184.72.254.190:8502


Tool #1: retrieve" knowledgeBaseId="kb_cloud_storage
tool_name=<retrieve" knowledgeBaseId="kb_cloud_storage> | invalid tool name pattern


Let me try a broad query in the knowledge base first to see if there are any relevant articles about slow upload speeds to storage:
Tool #2: retrieve


Unfortunately, I could not find any relevant knowledge base articles about troubleshooting slow upload speeds to storage. However, here are some common reasons that could cause slow uploads and steps to investigate:

1. Network bandwidth - Check if you are getting the expected network upload speeds from your internet service provider or corporate network. Slow networks will bottleneck upload performance.

2. Storage service throttling - Many cloud storage services will temporarily throttle upload speeds if you exceed certai